In [1]:
import anndata as ad
import pandas as pd
import shapely
import shapely.plotting as splot
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np

import ccf_plots as cplot

get_ipython().run_line_magic('matplotlib', 'inline') 

# Load brain3 data

In [2]:
obs = pd.read_parquet("/data/realigned-metadata/abc_realigned_metadata_thalamus-boundingbox.parquet")
ccf_label = 'parcellation_substructure'
nn_classes = [
    "31 OPC-Oligo",
    "30 Astro-Epen",
    "33 Vascular",
    "34 Immune",
]

In [3]:
# subset just the neurons
obs_neurons = obs[~obs['class'].isin(nn_classes)]

In [4]:
import abc_load as abc
ccf_df = pd.read_csv(
        abc.ABC_ROOT/"metadata/Allen-CCF-2020/20230630/parcellation_to_parcellation_term_membership.csv"
        )

In [5]:
th_zi_ind = np.hstack(
        (ccf_df.loc[ccf_df['parcellation_term_acronym']=='TH', 
                    'parcellation_index'].unique(),
             ccf_df.loc[ccf_df['parcellation_term_acronym']=='ZI', 
                    'parcellation_index'].unique())
)
len(th_zi_ind)

In [6]:
ccf_labels = ccf_df.pivot(index='parcellation_index', values='parcellation_term_acronym', columns='parcellation_term_set_name')
th_subregion_names = list(set(ccf_labels.loc[th_zi_ind, 'substructure']).difference(['TH-unassigned']))

# Per Subregion Metrics (neurons only)

In [7]:
# only look at the thalamic subregions as defined in th_subregion_names
obs_neurons_ccf = obs_neurons[obs_neurons[ccf_label].isin(th_subregion_names)]
                                # obs_neurons['parcellation_structure_realigned'].isin(th_subregion_names)]
# helps plotting etc by trimming categories
obs_neurons_ccf = obs_neurons_ccf.assign(
    **obs_neurons_ccf.select_dtypes(include=["category"]).apply(lambda x: x.cat.remove_unused_categories()).to_dict(orient='series'))

## Cell Type Richness

transcriptomic richness = \# of (clusters/supertypes/subclasses/classes) present in a given subregion of the thalamus

(Defined similarly to ecology's "species richness" = # of species in a given region)

In [8]:
# define a new df in which to store metrics
th_metrics = pd.DataFrame(index=th_subregion_names)

In [16]:
# generate lists of the clusters, supertypes, subclasses, and classes present
# in the thalamus data
th_clusters = obs_neurons_ccf['cluster'].drop_duplicates().tolist()
th_supertypes = obs_neurons_ccf['supertype'].drop_duplicates().tolist()
th_subclasses = obs_neurons_ccf['subclass'].drop_duplicates().tolist()
n_th_clusters = len(th_clusters)
n_th_supertypes = len(th_supertypes)
n_th_subclasses = len(th_subclasses)
print(f'{len(th_clusters)=}')
# print(f'{len(obs_neurons_ccf['cluster'].unique().tolist())}')
print(f'{len(th_supertypes)=}')
# print(f'{len(obs_neurons_ccf['supertype'].unique().tolist())}')
print(f'{len(th_subclasses)=}')
# print(f'{len(obs_neurons_ccf['subclass'].unique().tolist())}')

In [10]:
for subregion, df in obs_neurons_ccf.groupby(ccf_label, observed=True):
    # number of thalamic clusters, supertypes, subclasses in each subregion
    th_metrics.loc[subregion, 
                   'n_clusters'] = len(df['cluster'].drop_duplicates())
    th_metrics.loc[subregion, 
                   'n_supertypes'] = len(df['supertype'].drop_duplicates())
    th_metrics.loc[subregion, 
                   'n_subclasses'] = len(df['subclass'].drop_duplicates())

    # fraction of thalamic clusters, supertypes, subclasses in each 
    # subregion (denominator is total # of [tax. level] present in all of thalamus)
    th_metrics.loc[subregion, 
                   'frac_clusters'] = (len(df['cluster'].drop_duplicates()) 
                                       / n_th_clusters)
    th_metrics.loc[subregion,
                   'frac_supertypes'] = (len(df['supertype'].drop_duplicates())
                                         / n_th_supertypes)
    th_metrics.loc[subregion,
                   'frac_subclasses'] = (len(df['subclass'].drop_duplicates())
                                         / n_th_subclasses)
th_metrics['count'] = obs_neurons_ccf.groupby(ccf_label, observed=True).count().iloc[:,0]
th_metrics = th_metrics.join(obs_neurons_ccf.groupby(ccf_label, observed=True)
                            [['cluster', 'supertype', 'subclass']]
                            .aggregate(lambda x: len(x.unique())/len(x))
                            .rename(columns=lambda x: f"n_{x}_norm"))                          

In [11]:
# exlcude ZI because it's so much higher
ccf_names_noZI = [x for x in th_subregion_names if not 'ZI' in x]
th_metrics_noZI = th_metrics.loc[ccf_names_noZI]
th_metrics_sorted = th_metrics_noZI.sort_values(by='frac_supertypes', ascending=True)

# th_metrics_sorted

fig, ax1 = plt.subplots(figsize=(12,6))

# Plot the absolute values on the left y-axis
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['n_supertypes'], color='#5DA7E5', alpha=0)
ax1.set_ylabel('# of supertypes', color='k')
ax1.tick_params(axis='y', labelcolor='k')
ax1.set_xticks(th_metrics_sorted.index)
ax1.set_xticklabels(th_metrics_sorted.index, rotation=90)
ax1.set_ylim(0, th_metrics_sorted['n_supertypes'].max()*1.05)
plt.grid(visible=True, axis='y')

# Create a secondary y-axis on the right
ax2 = ax1.twinx()

# Plot the fraction values on the right y-axis
ax2.bar(th_metrics_sorted.index, th_metrics_sorted['frac_supertypes'], color='#5DA7E5', label='supertype')
ax2.set_ylabel('fraction of total supertypes (n='+str(n_th_supertypes)+')', color='k', rotation=270, labelpad=15)
ax2.set_ylim(0, th_metrics_sorted['frac_supertypes'].max()*1.05)
ax2.tick_params(axis='y', labelcolor='k')
ax2.legend()

# plt.xticks(rotation=90)
plt.xlabel('CCF thalamic subregions')
plt.title('supertypes per CCF thalamic subregion')

plt.show()

### relationship to region size?

In [14]:
th_metrics_sorted = th_metrics.sort_values(by='frac_supertypes', ascending=True)

fig, ax1 = plt.subplots(figsize=(12,6))

# Plot the absolute values on the left y-axis
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['frac_clusters'], 
            color='#F99D20', label='cluster', zorder=2) # orange
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['frac_supertypes'], 
            color='#5DA7E5', label='supertype', zorder=3) # blue
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['frac_subclasses'], 
            color='#1D8649', label='subclass', zorder=2) # green
ax1.set_ylabel('Cell type count (normalized to total)')
ax1.set_xticks(th_metrics_sorted.index)
ax1.set_xticklabels(th_metrics_sorted.index, rotation=90)
ax1.legend()
plt.grid(visible=True, axis='both', zorder=0, color='whitesmoke')
plt.ylim(0, 0.4)

In [34]:
import seaborn as sns
sns.regplot(th_metrics_noZI, x='count', y='frac_clusters', color='#F99D20', label='cluster') 

In [35]:

sns.regplot(th_metrics_noZI, x='count', y='frac_supertypes', color='#5DA7E5', label='cluster') 

In [36]:

sns.regplot(th_metrics_noZI, x='count', y='frac_subclasses', color='#1D8649', label='cluster') 

In [16]:
th_metrics_sorted = th_metrics.sort_values(by='n_supertype_norm', ascending=True)

fig, ax1 = plt.subplots(figsize=(12,6))

# Plot the absolute values on the left y-axis
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['n_cluster_norm'], 
            color='#F99D20', label='cluster', zorder=2) # orange
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['n_supertype_norm'], 
            color='#5DA7E5', label='supertype', zorder=3) # blue
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['n_subclass_norm'], 
            color='#1D8649', label='subclass', zorder=2) # green
ax1.set_ylabel('Cell type count (normalized to cell count)')
ax1.set_xticks(th_metrics_sorted.index)
ax1.set_xticklabels(th_metrics_sorted.index, rotation=90)
ax1.legend()
plt.grid(visible=True, axis='both', zorder=0, color='whitesmoke')

### CCF colored by value

In [17]:
imdata = abc.get_ccf_labels_image(resampled=True).astype(int)
# x, y, z orientation
imdata.shape

In [18]:
# parcellation_index to acronym
substructure_index = ccf_df.query("parcellation_term_set_name=='substructure'").set_index('parcellation_index')['parcellation_term_acronym'].to_dict()


In [19]:
sections = [7.2] 
print('Supertypes')
cplot.plot_metrics_ccf_raster(imdata, th_metrics['n_supertypes'], sections, substructure_index,
                            cmap='Blues', cb_label='number of supertypes in subregion')

In [20]:

print('Clusters')
cplot.plot_metrics_ccf_raster(imdata, th_metrics['n_supertypes'], sections, substructure_index,
                       cmap='Oranges', cb_label='number of clusters in subregion')

## Cell Type Diversity

### Shannon Diversity Index

In [21]:
# Shannon diversity index, normalized to (0,1)
# 1 = high diversity (clusters found in equal proportions)
# 0 = low diversity (some clusters found in higher proportions in subregion)

def calc_shannon_index(obs_col, n_total_categories):
    # calculate proportion of each category
    cateogory_counts = obs_col.value_counts()
    cateogory_counts = cateogory_counts[cateogory_counts>0] # cleanup zeros so log() doesn't throw warnings
    p = cateogory_counts / cateogory_counts.sum()
    
    # calculate shannon diversity index
    shannon_ind = (-1)*((p * np.log2(p)).sum())
    
    # normalize by log2 of number of categories to limit index to range (0, 1)
    shannon_norm = shannon_ind / np.log2(n_total_categories)
    
    return shannon_norm

In [22]:
# clusters
for subregion, df in obs_neurons_ccf.groupby(ccf_label, observed=True):
    th_metrics.loc[subregion, 
                   'shannon_index_clusters'] = calc_shannon_index(df['cluster'], 
                                                                  n_th_clusters)
    th_metrics.loc[subregion, 
                   'shannon_index_supertypes'] = calc_shannon_index(df['supertype'], 
                                                                  n_th_clusters)
    th_metrics.loc[subregion, 
                   'shannon_index_subclasses'] = calc_shannon_index(df['subclass'], 
                                                                  n_th_clusters)

In [23]:
# exlcude ZI because it's so much higher
# th_metrics_noZI = th_metrics.loc[ccf_names_noZI]

# sort by cluster shannon index
th_metrics_sorted = th_metrics.sort_values(by='shannon_index_supertypes', ascending=True)
# th_metrics_sorted

fig, ax1 = plt.subplots(figsize=(12,6))

# Plot the absolute values on the left y-axis
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['shannon_index_clusters'], 
            color='#F99D20', label='cluster', zorder=2) # orange
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['shannon_index_supertypes'], 
            color='#5DA7E5', label='supertype', zorder=3) # blue
ax1.scatter(th_metrics_sorted.index, th_metrics_sorted['shannon_index_subclasses'], 
            color='#1D8649', label='subclass', zorder=2) # green

# FFE700 yellow
ax1.set_ylabel('Shannon Diversity Index', color='k')
ax1.tick_params(axis='y', labelcolor='k')
ax1.set_xticks(th_metrics_sorted.index)
ax1.set_xticklabels(th_metrics_sorted.index, rotation=90)
ax1.legend()
plt.grid(visible=True, axis='both', zorder=0, color='whitesmoke')

# plt.xticks(rotation=90)
plt.xlabel('CCF thalamic subregions')
plt.title('Shannon Diversity Index')

plt.show()

In [24]:
sections = [7.2] 
label = 'Shannon Index - clusters'
cplot.plot_metrics_ccf_raster(imdata, th_metrics['shannon_index_clusters'], sections, substructure_index,
                            cmap='Oranges', cb_label=label)

In [25]:

label = 'Shannon Index - supertypes'
cplot.plot_metrics_ccf_raster(imdata, th_metrics['shannon_index_supertypes'], sections, substructure_index,
                            cmap='Blues', cb_label=label)

In [26]:

label = 'Shannon Index - subclasses'
cplot.plot_metrics_ccf_raster(imdata, th_metrics['shannon_index_subclasses'], sections, substructure_index,
                            cmap='Greens', cb_label=label)

### Proportions as stacked bar graphs

In [27]:
# for each subregion, count the proportion of cells in each cluster/supertype/subclass
th_subregions = th_subregion_names
ccf_cluster_counts = pd.DataFrame(index=th_subregions, columns=th_clusters)
ccf_supertype_counts = pd.DataFrame(index=th_subregions, columns=th_supertypes)
ccf_subclass_counts = pd.DataFrame(index=th_subregions, columns=th_subclasses)
for subregion, df in obs_neurons_ccf.groupby(ccf_label, observed=True):
    # clusters
    cluster_counts = df['cluster'].value_counts()
    ccf_cluster_counts.loc[subregion, 
                           cluster_counts.index] = cluster_counts.values / len(df)

    # supertypes
    supertype_counts = df['supertype'].value_counts()
    ccf_supertype_counts.loc[subregion, 
                             supertype_counts.index] = supertype_counts.values / len(df)

    # subclasses
    subclass_counts = df['subclass'].value_counts()
    ccf_subclass_counts.loc[subregion, 
                            subclass_counts.index] = subclass_counts.values / len(df)

#### TODO: Implement barchart w/ plotly

which will enable hovering over the bars to display the subclass/supertype/cluster to which it belongs

In [28]:
# display as a stacked bar graph
from colorcet import glasbey_light

def plot_stacked_barplot(df, taxonomy_level, legend=True):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,6))
    colors = sns.color_palette(glasbey_light, n_colors=len(df.columns))
    df.plot(kind='bar', stacked=True, ax=axes, legend=legend, color=colors)
    if legend:
        axes.legend(loc='upper left', bbox_to_anchor=(0.05, -0.3), ncol=4)
    axes.set_xticklabels(df.index, rotation=90)
    axes.set_xlabel('CCF subregion')
    # axes.set_yticks([])
    axes.set_ylabel('proportion of cells in unique '+taxonomy_level)

    fig.subplots_adjust(hspace=0.1)
    
    # add text
    for i, subregion in enumerate(df.index):
        n_nonzero = (df.loc[subregion]>0).sum()
        axes.text(i, 1.01, n_nonzero, horizontalalignment='center')
    plt.show()

#### Subclasses

In [29]:
# First sort by the Shannon Diversity Index from th_metrics
th_metrics_sorted_by_sdi = th_metrics.sort_values(by='shannon_index_subclasses', ascending=True)

# Use sorted list of subregions to plot in order from least diverse to most diverse
plot_stacked_barplot(ccf_subclass_counts.loc[th_metrics_sorted_by_sdi.index], 
                     'subclasses', legend=False)

#### Supertypes

In [30]:
# First sort by the Shannon Diversity Index from th_metrics
th_metrics_sorted_by_sdi = th_metrics.sort_values(by='shannon_index_supertypes', ascending=True)

# Use sorted list of subregions to plot in order from least diverse to most diverse
plot_stacked_barplot(ccf_supertype_counts.loc[th_metrics_sorted_by_sdi.index], 
                     'supertypes', legend=False)

#### Clusters

In [31]:
# First sort by the Shannon Diversity Index from th_metrics
th_metrics_sorted_by_sdi = th_metrics.sort_values(by='shannon_index_clusters', ascending=True)

# Use sorted list of subregions to plot in order from least diverse to most diverse
plot_stacked_barplot(ccf_cluster_counts.loc[th_metrics_sorted_by_sdi.index], 
                     'clusters', legend=False)

# Per Cluster/Supertype/Subclass Metrics (neurons only)

## TODO: Dispersion of clusters/supertypes across nuclei

i.e. grab a cluster that dominates in one nucleus (e.g. 1133 in MD) and ask how dispersed it is across the whole thalamus. Ideally, a linear metric, such that you could ask what the dispersion is for all clusters to see which ones seem to be more compact and which are more shared

I *think* this is going to have to be done on hemispheres, rather than the whole brain. Most existing tools I'm finding are not able to account for symmetries.

Likely going to borrow from some of ArcGIS's standard toolboxes
- https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/h-how-standard-distance-spatial-statistic-works.htm

Or other satellite imagery methods:
- https://www.tandfonline.com/doi/full/10.1080/17538947.2018.1474957

# Neighborhood metrics

Single neuron tracing in the MD (shown in weekly U19 on Fri 7/28/23) illustrates two things:
1. Some *neighboring* soma have *distinct* projection patterns
2. Some *distant* soma have *very similar* projection patterns

Jayaram's interpretation is that \#1 is surprising given gene expression in the MD.

But I don't think that's the correct interpretation, especially in the MD! 
- MD contains a large diversity of cell types, many of which show salt-and-pepper sprinkling throughout the MD.
- At the very least, we don't have our MERFISH data analyzed in the right way to say whether or not the single neuron data is surprising/unexpected
- They really need their mFISH data to say this for certain

## TODO: most likely/common neighboring cell type metric

"most common / most likely neighbor" metrics for cells
- How likely are you to find a cell with (a) the same or (b) a different transcriptomic type right next to your cell of interest?
- What's the average fraction of nearest neighbors that are the same? different?
- How many non-self cell types are represented in the nearest neighbors?
- Do these metrics change from nucleus to nucleus? i.e. do cells in the MD have more diverse neighbors than, say, AD? (shannon index might suggest that they do)